In [1]:
import os
import sys
import glob
import numpy as np
from skimage import io
import librosa
import librosa.display
import matplotlib.pyplot as plt
#from sklearn import datasets
from sklearn import utils
import cv2
from PIL import Image

In [43]:
IMAGE_SIZE_X = 500
IMAGE_SIZE_Y = 128
COLOR_BYTE = 3
CATEGORY_NUM = 6
threshhold = 80

In [67]:
def load_split_bin(path):
    # ファイル一覧を取得
    files = glob.glob(os.path.join(path, '*/*.wav'))

    # イメージとラベル領域を確保
    images = np.ndarray((len(files), IMAGE_SIZE_X, IMAGE_SIZE_Y), dtype = np.uint8)
    #images = np.ndarray((len(files), IMAGE_SIZE_X, IMAGE_SIZE_Y)
    #                   , dtype = np.uint8)

    labels = np.ndarray(len(files), dtype=np.int)
    imagess =[]
    # イメージとラベルを読み込み
    for idx, file in enumerate(files):
        x = 0
        #オーディオファイルの読み込み
        data, fs = librosa.audio.load(file)
        #フーリエ変換
        D = librosa.stft(data)
        #数値をGAIN表記に変換
        log_power = librosa.amplitude_to_db(np.abs(D), ref=np.max)
        #無音部分の削除
        log_powern = (log_power[:, np.any(log_power > -50, axis=0)])
        #メルスペクトログラム配列に変換
        melsp = librosa.feature.melspectrogram(S=log_powern,n_mels=128)

        imagess.append(melsp[:,:500])
        label = os.path.split(os.path.dirname(file))[-1]
        labels[idx] = int(label)
    images = np.array(imagess)

        # scikit-learn の他のデータセットの形式に合わせる
    flat_data = images.reshape((-1, IMAGE_SIZE_X * IMAGE_SIZE_Y * COLOR_BYTE))
    images = flat_data.view()
    return utils.Bunch(data=images,
                 target=labels.astype(np.int),
                 target_names=np.arange(CATEGORY_NUM),
                 images=images,
                 DESCR=None)


In [68]:
d = load_split_bin("./dataset/")

ValueError: could not broadcast input array from shape (128,207) into shape (128)

In [46]:
print(d)

{'data': array([[-5.741701 , -5.254249 , -4.680145 , ..., -7.1283674, -7.1330075,
        -7.330424 ],
       [-6.473911 , -5.820076 , -5.6906385, ..., -7.4295387, -7.4225554,
        -7.422807 ],
       [-5.7534437, -6.455452 , -6.947953 , ..., -7.231885 , -7.1205816,
        -6.9177485],
       ...,
       [-5.289723 , -5.5733805, -5.7461853, ..., -7.2285175, -6.205873 ,
        -5.8667355],
       [-5.871008 , -5.5052285, -5.1846304, ..., -7.40518  , -7.389081 ,
        -7.3251863],
       [-6.171636 , -5.8390884, -5.2854133, ..., -7.2374353, -7.1242523,
        -7.1249638]], dtype=float32), 'target': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0

In [63]:
print(len(d.target))
print(len(d.images))

299
369


In [61]:
flag_0_1 = (d.target == 0)+(d.target == 1)

In [62]:
voice = d.images[flag_0_1]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 369 but corresponding boolean dimension is 299

In [49]:
labels = d.target

In [52]:
voice = voice.reshape(voice.shape[0], -1)

In [54]:
from sklearn import tree
n_samples = len(voice)

In [55]:
train_size = int(n_samples * 3/5)

In [56]:
classifier = tree.DecisionTreeClassifier()

In [57]:
classifier.fit(voice[:train_size], labels[:train_size])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [58]:
from sklearn import metrics

In [59]:
expected = labels[train_size:]
predicted = classifier.predict(voice[train_size:])

In [60]:
print("Accuracy:\n",metrics.accuracy_score(expected, predicted))

ValueError: Found input variables with inconsistent numbers of samples: [78, 148]